## Demand Estimation Exercise

In this assignment, you will learn how to simulate market data and estimate simple random coefficient demand models. The assignment should serve as an introduction to the structural estimation in heterogenous product demand models.

Let's first define the model.


In [1]:
import simulate
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

In [2]:
n_firms = 10
n_const = 1000
n_caract = 4
T = 10


market = simulate.Market_Data(n_firms, n_const, n_caract, T)
print(market)

Market with 10 firms and 1000 consumers over 10 time periods. 
 Firms sell differentiated product which have 4 product characteristics


This market is characterized y the following parameters that are unknown to the econometrician:


In [3]:
df = market.generate_simulated_data()
print(df)

    market_ids  firm_ids  market_share     price      cost        xi  \
0            1         1      0.000005  1.150613  0.416911 -0.340930   
1            1         2      0.103753  2.552354  1.186894 -0.227252   
2            1         3      0.042450  3.575963  1.779830 -0.594663   
3            1         4      0.664267  2.376352  0.881423  0.006663   
4            1         5      0.024970  4.921845  1.633395 -0.049778   
..         ...       ...           ...       ...       ...       ...   
95          10         6      0.016094  2.928746  1.001510  0.129835   
96          10         7      0.226754  2.552623  0.962702 -0.034956   
97          10         8      0.002245  3.117963  1.001981  0.203453   
98          10         9      0.030695  2.033158  0.911643 -0.119997   
99          10        10      0.389349  1.788144  0.288351  0.315233   

       char1     char2     char3     char4  
0   3.278645  1.193333  3.995015  1.002434  
1   3.509068  3.508631  1.303348  5.749185  
